<a href="https://colab.research.google.com/github/AnnaKraim/ML/blob/main/fifa_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
import json
with open('fifa_data.json', 'r') as f:
  data = json.loads(f.read())

In [ ]:
df = pd.DataFrame(data['matches'])
df.head()

,venue,location,status,time,fifa_id,weather,attendance,officials,stage_name,home_team_country,away_team_country,datetime,winner,winner_code,home_team,away_team,home_team_events,away_team_events,home_team_statistics,away_team_statistics,last_event_update_at,last_score_update_at
0,Moscow,Luzhniki Stadium,completed,full-time,300331503,"{'humidity': '43', 'temp_celsius': '17', 'temp...",78011,"[Nestor PITANA, Hernan MAIDANA, Juan Pablo BEL...",First stage,Russia,Saudi Arabia,2018-06-14T15:00:00Z,Russia,RUS,"{'country': 'Russia', 'code': 'RUS', 'goals': ...","{'country': 'Saudi Arabia', 'code': 'KSA', 'go...","[{'id': 5, 'type_of_event': 'goal', 'player': ...","[{'id': 12, 'type_of_event': 'substitution-in'...","{'country': 'Russia', 'attempts_on_goal': 13, ...","{'country': 'Saudi Arabia', 'attempts_on_goal'...",2018-06-21T02:36:44Z,None
1,Ekaterinburg,Ekaterinburg Arena,completed,full-time,300353632,"{'humidity': '42', 'temp_celsius': '14', 'temp...",27015,"[Bjorn KUIPERS, Sander VAN ROEKEL, Erwin ZEINS...",First stage,Egypt,Uruguay,2018-06-15T12:00:00Z,Uruguay,URU,"{'country': 'Egypt', 'code': 'EGY', 'goals': 0...","{'country': 'Uruguay', 'code': 'URU', 'goals':...","[{'id': 14, 'type_of_event': 'substitution-in'...","[{'id': 15, 'type_of_event': 'substitution-in'...","{'country': 'Egypt', 'attempts_on_goal': 8, 'o...","{'country': 'Uruguay', 'attempts_on_goal': 14,...",2018-06-21T02:37:25Z,None
2,St. Petersburg,Saint Petersburg Stadium,completed,full-time,300331526,"{'humidity': '37', 'temp_celsius': '25', 'temp...",62548,"[Cuneyt CAKIR, Bahattin DURAN, Tarik ONGUN, Se...",First stage,Morocco,Iran,2018-06-15T15:00:00Z,Iran,IRN,"{'country': 'Morocco', 'code': 'MAR', 'goals':...","{'country': 'Iran', 'code': 'IRN', 'goals': 1,...","[{'id': 23, 'type_of_event': 'yellow-card', 'p...","[{'id': 22, 'type_of_event': 'yellow-card', 'p...","{'country': 'Morocco', 'attempts_on_goal': 13,...","{'country': 'Iran', 'attempts_on_goal': 8, 'on...",2018-06-21T02:36:30Z,None
3,Sochi,Fisht Stadium,completed,full-time,300331524,"{'humidity': '74', 'temp_celsius': '24', 'temp...",43866,"[Gianluca ROCCHI, Elenito DI LIBERATORE, Mauro...",First stage,Portugal,Spain,2018-06-15T18:00:00Z,Draw,Draw,"{'country': 'Portugal', 'code': 'POR', 'goals'...","{'country': 'Spain', 'code': 'ESP', 'goals': 3...","[{'id': 36, 'type_of_event': 'goal-penalty', '...","[{'id': 37, 'type_of_event': 'yellow-card', 'p...","{'country': 'Portugal', 'attempts_on_goal': 8,...","{'country': 'Spain', 'attempts_on_goal': 12, '...",2018-06-21T02:36:18Z,2018-06-15T19:52:38Z
4,Kazan,Kazan Arena,completed,full-time,300331533,"{'humidity': '32', 'temp_celsius': '19', 'temp...",41279,"[Andres CUNHA, Nicolas TARAN, Mauricio ESPINOS...",First stage,France,Australia,2018-06-16T10:00:00Z,France,FRA,"{'country': 'France', 'code': 'FRA', 'goals': ...","{'country': 'Australia', 'code': 'AUS', 'goals...","[{'id': 52, 'type_of_event': 'goal-penalty', '...","[{'id': 50, 'type_of_event': 'yellow-card', 'p...","{'country': 'France', 'attempts_on_goal': 12, ...","{'country': 'Australia', 'attempts_on_goal': 4...",2018-06-21T02:42:39Z,2018-06-16T11:52:21Z


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 22 columns):
venue                   64 non-null object
location                64 non-null object
status                  64 non-null object
time                    64 non-null object
fifa_id                 64 non-null object
weather                 64 non-null object
attendance              64 non-null object
officials               64 non-null object
stage_name              64 non-null object
home_team_country       64 non-null object
away_team_country       64 non-null object
datetime                64 non-null object
winner                  64 non-null object
winner_code             64 non-null object
home_team               64 non-null object
away_team               64 non-null object
home_team_events        64 non-null object
away_team_events        64 non-null object
home_team_statistics    64 non-null object
away_team_statistics    64 non-null object
last_event_update_at    64 non-null o

In [ ]:
df['attendance'] = df['attendance'].astype(int)
df['attendance'].sum()

3031768

In [ ]:
# df['weather'][0]

def extract_cels(x):
  return int(x['temp_celsius'])

df['temp_celsius'] = df['weather'].apply(extract_cels)

In [ ]:
df.head()

,venue,location,status,time,fifa_id,weather,attendance,officials,stage_name,home_team_country,away_team_country,datetime,winner,winner_code,home_team,away_team,home_team_events,away_team_events,home_team_statistics,away_team_statistics,last_event_update_at,last_score_update_at,temp_celsius
0,Moscow,Luzhniki Stadium,completed,full-time,300331503,"{'humidity': '43', 'temp_celsius': '17', 'temp...",78011,"[Nestor PITANA, Hernan MAIDANA, Juan Pablo BEL...",First stage,Russia,Saudi Arabia,2018-06-14T15:00:00Z,Russia,RUS,"{'country': 'Russia', 'code': 'RUS', 'goals': ...","{'country': 'Saudi Arabia', 'code': 'KSA', 'go...","[{'id': 5, 'type_of_event': 'goal', 'player': ...","[{'id': 12, 'type_of_event': 'substitution-in'...","{'country': 'Russia', 'attempts_on_goal': 13, ...","{'country': 'Saudi Arabia', 'attempts_on_goal'...",2018-06-21T02:36:44Z,None,17
1,Ekaterinburg,Ekaterinburg Arena,completed,full-time,300353632,"{'humidity': '42', 'temp_celsius': '14', 'temp...",27015,"[Bjorn KUIPERS, Sander VAN ROEKEL, Erwin ZEINS...",First stage,Egypt,Uruguay,2018-06-15T12:00:00Z,Uruguay,URU,"{'country': 'Egypt', 'code': 'EGY', 'goals': 0...","{'country': 'Uruguay', 'code': 'URU', 'goals':...","[{'id': 14, 'type_of_event': 'substitution-in'...","[{'id': 15, 'type_of_event': 'substitution-in'...","{'country': 'Egypt', 'attempts_on_goal': 8, 'o...","{'country': 'Uruguay', 'attempts_on_goal': 14,...",2018-06-21T02:37:25Z,None,14
2,St. Petersburg,Saint Petersburg Stadium,completed,full-time,300331526,"{'humidity': '37', 'temp_celsius': '25', 'temp...",62548,"[Cuneyt CAKIR, Bahattin DURAN, Tarik ONGUN, Se...",First stage,Morocco,Iran,2018-06-15T15:00:00Z,Iran,IRN,"{'country': 'Morocco', 'code': 'MAR', 'goals':...","{'country': 'Iran', 'code': 'IRN', 'goals': 1,...","[{'id': 23, 'type_of_event': 'yellow-card', 'p...","[{'id': 22, 'type_of_event': 'yellow-card', 'p...","{'country': 'Morocco', 'attempts_on_goal': 13,...","{'country': 'Iran', 'attempts_on_goal': 8, 'on...",2018-06-21T02:36:30Z,None,25
3,Sochi,Fisht Stadium,completed,full-time,300331524,"{'humidity': '74', 'temp_celsius': '24', 'temp...",43866,"[Gianluca ROCCHI, Elenito DI LIBERATORE, Mauro...",First stage,Portugal,Spain,2018-06-15T18:00:00Z,Draw,Draw,"{'country': 'Portugal', 'code': 'POR', 'goals'...","{'country': 'Spain', 'code': 'ESP', 'goals': 3...","[{'id': 36, 'type_of_event': 'goal-penalty', '...","[{'id': 37, 'type_of_event': 'yellow-card', 'p...","{'country': 'Portugal', 'attempts_on_goal': 8,...","{'country': 'Spain', 'attempts_on_goal': 12, '...",2018-06-21T02:36:18Z,2018-06-15T19:52:38Z,24
4,Kazan,Kazan Arena,completed,full-time,300331533,"{'humidity': '32', 'temp_celsius': '19', 'temp...",41279,"[Andres CUNHA, Nicolas TARAN, Mauricio ESPINOS...",First stage,France,Australia,2018-06-16T10:00:00Z,France,FRA,"{'country': 'France', 'code': 'FRA', 'goals': ...","{'country': 'Australia', 'code': 'AUS', 'goals...","[{'id': 52, 'type_of_event': 'goal-penalty', '...","[{'id': 50, 'type_of_event': 'yellow-card', 'p...","{'country': 'France', 'attempts_on_goal': 12, ...","{'country': 'Australia', 'attempts_on_goal': 4...",2018-06-21T02:42:39Z,2018-06-16T11:52:21Z,19



#### 2. 
Найдите среднюю температуру воздуха в градусах цельсия с точностью до 2 знаков после запятой.



In [ ]:
df['temp_celsius'].mean()

23.75

3.
Найдите самый активный матч

In [ ]:
df['event_n'] = df['home_team_events'].apply(len) + df['away_team_events'].apply(len)

# df.loc[df['event_n'].idxmax()]
df[df['event_n'] == df['event_n'].max()]

,venue,location,status,time,fifa_id,weather,attendance,officials,stage_name,home_team_country,away_team_country,datetime,winner,winner_code,home_team,away_team,home_team_events,away_team_events,home_team_statistics,away_team_statistics,last_event_update_at,last_score_update_at,temp_celsius,event_n
48,Kazan,Kazan Arena,completed,full-time,300331537,"{'humidity': '33', 'temp_celsius': '28', 'temp...",42873,"[Alireza FAGHANI, Reza SOKHANDAN, Mohammadreza...",Round of 16,France,Argentina,2018-06-30T14:00:00Z,France,FRA,"{'country': 'France', 'code': 'FRA', 'goals': ...","{'country': 'Argentina', 'code': 'ARG', 'goals...","[{'id': 877, 'type_of_event': 'goal-penalty', ...","[{'id': 876, 'type_of_event': 'yellow-card', '...","{'country': 'France', 'attempts_on_goal': 9, '...","{'country': 'Argentina', 'attempts_on_goal': 9...",2018-06-30T17:16:20Z,2018-06-30T17:16:20Z,28,27
55,Moscow,Spartak Stadium,completed,full-time,300331542,"{'humidity': '61', 'temp_celsius': '18', 'temp...",44190,"[Mark GEIGER, Joe FLETCHER, Frank ANDERSON, Ma...",Round of 16,Colombia,England,2018-07-03T18:00:00Z,England,ENG,"{'country': 'Colombia', 'code': 'COL', 'goals'...","{'country': 'England', 'code': 'ENG', 'goals':...","[{'id': 1012, 'type_of_event': 'yellow-card', ...","[{'id': 1015, 'type_of_event': 'yellow-card', ...","{'country': 'Colombia', 'attempts_on_goal': 14...","{'country': 'England', 'attempts_on_goal': 16,...",2018-07-03T20:53:13Z,2018-07-03T20:53:13Z,18,27
59,Sochi,Fisht Stadium,completed,full-time,300331504,"{'humidity': '70', 'temp_celsius': '24', 'temp...",44287,"[Sandro RICCI, Emerson DE CARVALHO, Marcelo VA...",Quarter-finals,Russia,Croatia,2018-07-07T18:00:00Z,Croatia,CRO,"{'country': 'Russia', 'code': 'RUS', 'goals': ...","{'country': 'Croatia', 'code': 'CRO', 'goals':...","[{'id': 1094, 'type_of_event': 'goal', 'player...","[{'id': 1095, 'type_of_event': 'yellow-card', ...","{'country': 'Russia', 'attempts_on_goal': 13, ...","{'country': 'Croatia', 'attempts_on_goal': 17,...",2018-07-07T20:51:30Z,2018-07-07T20:51:30Z,24,27


##### Найдите команду, которая забила больше всех голов за все время турнира

In [ ]:
# выделяем голы
df['home_team_goals'] = df['home_team'].apply(lambda x:x['goals'])
df['away_team_goals'] = df['away_team'].apply(lambda x:x['goals'])


Делаем с помощью обычного питона и цикла

In [ ]:
# найти команду которая сделала больше всех голов ?

from collections import Counter

c = Counter()

for i, x in df.iterrows():
  c[x['home_team_country']] += x['home_team_goals']
  c[x['away_team_country']] += x['away_team_goals']

c.most_common()

[('Belgium', 16),
 ('France', 14),
 ('Croatia', 14),
 ('England', 12),
 ('Russia', 11),
 ('Brazil', 8),
 ('Uruguay', 7),
 ('Spain', 7),
 ('Portugal', 6),
 ('Argentina', 6),
 ('Sweden', 6),
 ('Colombia', 6),
 ('Japan', 6),
 ('Switzerland', 5),
 ('Tunisia', 5),
 ('Senegal', 4),
 ('Denmark', 3),
 ('Nigeria', 3),
 ('Mexico', 3),
 ('Korea Republic', 3),
 ('Saudi Arabia', 2),
 ('Egypt', 2),
 ('Morocco', 2),
 ('Iran', 2),
 ('Australia', 2),
 ('Iceland', 2),
 ('Peru', 2),
 ('Costa Rica', 2),
 ('Serbia', 2),
 ('Germany', 2),
 ('Panama', 2),
 ('Poland', 2)]

С помощью pandas groupby

In [ ]:
# Russia -> [m1, m2, m3]
# Belgium -> [m10, m21]
# ...

h_goals = df.groupby('home_team_country')['home_team_goals'].sum()
a_goals = df.groupby('away_team_country')['away_team_goals'].sum()
(h_goals + a_goals).sort_values(ascending=False)

home_team_country
Belgium           16
Croatia           14
France            14
England           12
Russia            11
Brazil             8
Uruguay            7
Spain              7
Japan              6
Colombia           6
Argentina          6
Portugal           6
Sweden             6
Switzerland        5
Tunisia            5
Senegal            4
Denmark            3
Korea Republic     3
Mexico             3
Nigeria            3
Iran               2
Iceland            2
Germany            2
Panama             2
Saudi Arabia       2
Egypt              2
Poland             2
Costa Rica         2
Serbia             2
Morocco            2
Australia          2
Peru               2
dtype: int64

Найдите город проведения с самой большой средней температурой за все матчи

In [ ]:
df.groupby('location')['temp_celsius'].mean()

location
Ekaterinburg Arena          19.500000
Fisht Stadium               25.166667
Kaliningrad Stadium         17.750000
Kazan Arena                 23.166667
Luzhniki Stadium            22.285714
Mordovia Arena              22.000000
Nizhny Novgorod Stadium     23.166667
Rostov Arena                29.000000
Saint Petersburg Stadium    20.428571
Samara Arena                27.333333
Spartak Stadium             23.800000
Volgograd Arena             31.750000
Name: temp_celsius, dtype: float64

Какая была самая популярная расстановка на поле у  команд-хозяев?

In [ ]:
df['home_team_statistics'].apply(lambda x:x['tactics']).value_counts()

4-2-3-1    33
4-4-2      10
3-4-3       6
3-5-2       5
4-3-3       4
4-5-1       3
5-4-1       1
3-4-2-1     1
4-3-2-1     1
Name: home_team_statistics, dtype: int64

Найдите команду, которая делала меньше всего замен в среднем за игру. 

Можно ли это решить так же, как и с голами ?

In [ ]:
# выделяем замены

def count_substitutions(ev_arr):
  s = 0
  for v in ev_arr:
    if v['type_of_event'].startswith('substitution'):
      s += 1
  return s

df['home_team_subst'] = df['home_team_events'].apply(count_substitutions)
df['away_team_subst'] = df['away_team_events'].apply(count_substitutions)

df.head()
# df['home_team_goals'] = df['home_team'].apply(lambda x:x['goals'])
# df['away_team_goals'] = df['away_team'].apply(lambda x:x['goals'])


,venue,location,status,time,fifa_id,weather,attendance,officials,stage_name,home_team_country,away_team_country,datetime,winner,winner_code,home_team,away_team,home_team_events,away_team_events,home_team_statistics,away_team_statistics,last_event_update_at,last_score_update_at,temp_celsius,event_n,home_team_goals,away_team_goals,home_team_subst,away_team_subst
0,Moscow,Luzhniki Stadium,completed,full-time,300331503,"{'humidity': '43', 'temp_celsius': '17', 'temp...",78011,"[Nestor PITANA, Hernan MAIDANA, Juan Pablo BEL...",First stage,Russia,Saudi Arabia,2018-06-14T15:00:00Z,Russia,RUS,"{'country': 'Russia', 'code': 'RUS', 'goals': ...","{'country': 'Saudi Arabia', 'code': 'KSA', 'go...","[{'id': 5, 'type_of_event': 'goal', 'player': ...","[{'id': 12, 'type_of_event': 'substitution-in'...","{'country': 'Russia', 'attempts_on_goal': 13, ...","{'country': 'Saudi Arabia', 'attempts_on_goal'...",2018-06-21T02:36:44Z,None,17,20,5,0,6,7
1,Ekaterinburg,Ekaterinburg Arena,completed,full-time,300353632,"{'humidity': '42', 'temp_celsius': '14', 'temp...",27015,"[Bjorn KUIPERS, Sander VAN ROEKEL, Erwin ZEINS...",First stage,Egypt,Uruguay,2018-06-15T12:00:00Z,Uruguay,URU,"{'country': 'Egypt', 'code': 'EGY', 'goals': 0...","{'country': 'Uruguay', 'code': 'URU', 'goals':...","[{'id': 14, 'type_of_event': 'substitution-in'...","[{'id': 15, 'type_of_event': 'substitution-in'...","{'country': 'Egypt', 'attempts_on_goal': 8, 'o...","{'country': 'Uruguay', 'attempts_on_goal': 14,...",2018-06-21T02:37:25Z,None,14,15,0,1,6,6
2,St. Petersburg,Saint Petersburg Stadium,completed,full-time,300331526,"{'humidity': '37', 'temp_celsius': '25', 'temp...",62548,"[Cuneyt CAKIR, Bahattin DURAN, Tarik ONGUN, Se...",First stage,Morocco,Iran,2018-06-15T15:00:00Z,Iran,IRN,"{'country': 'Morocco', 'code': 'MAR', 'goals':...","{'country': 'Iran', 'code': 'IRN', 'goals': 1,...","[{'id': 23, 'type_of_event': 'yellow-card', 'p...","[{'id': 22, 'type_of_event': 'yellow-card', 'p...","{'country': 'Morocco', 'attempts_on_goal': 13,...","{'country': 'Iran', 'attempts_on_goal': 8, 'on...",2018-06-21T02:36:30Z,None,25,20,0,1,7,8
3,Sochi,Fisht Stadium,completed,full-time,300331524,"{'humidity': '74', 'temp_celsius': '24', 'temp...",43866,"[Gianluca ROCCHI, Elenito DI LIBERATORE, Mauro...",First stage,Portugal,Spain,2018-06-15T18:00:00Z,Draw,Draw,"{'country': 'Portugal', 'code': 'POR', 'goals'...","{'country': 'Spain', 'code': 'ESP', 'goals': 3...","[{'id': 36, 'type_of_event': 'goal-penalty', '...","[{'id': 37, 'type_of_event': 'yellow-card', 'p...","{'country': 'Portugal', 'attempts_on_goal': 8,...","{'country': 'Spain', 'attempts_on_goal': 12, '...",2018-06-21T02:36:18Z,2018-06-15T19:52:38Z,24,20,3,3,6,6
4,Kazan,Kazan Arena,completed,full-time,300331533,"{'humidity': '32', 'temp_celsius': '19', 'temp...",41279,"[Andres CUNHA, Nicolas TARAN, Mauricio ESPINOS...",First stage,France,Australia,2018-06-16T10:00:00Z,France,FRA,"{'country': 'France', 'code': 'FRA', 'goals': ...","{'country': 'Australia', 'code': 'AUS', 'goals...","[{'id': 52, 'type_of_event': 'goal-penalty', '...","[{'id': 50, 'type_of_event': 'yellow-card', 'p...","{'country': 'France', 'attempts_on_goal': 12, ...","{'country': 'Australia', 'attempts_on_goal': 4...",2018-06-21T02:42:39Z,2018-06-16T11:52:21Z,19,20,2,1,6,6


In [ ]:
# срзнач (sum(10чисел) / 10) + срзнач(sum(1числа) / 1) != срзнач(11 чисел / 11)


h_subst = df.groupby('home_team_country')['home_team_subst'].agg(['sum', 'size'])
# h_subst
a_subst = df.groupby('away_team_country')['away_team_subst'].agg(['sum', 'size'])

mean_subst = h_subst + a_subst

mean_subst['mean'] = mean_subst['sum'] / mean_subst['size']

mean_subst.sort_values(by='mean', ascending=False)
# (h_subst + a_subst).sort_values(ascending=False)

,sum,size,mean
home_team_country,,,
Croatia,50,7,7.142857
Morocco,21,3,7.000000
Russia,34,5,6.800000
Nigeria,20,3,6.666667
Iran,20,3,6.666667
Colombia,26,4,6.500000
Spain,26,4,6.500000
Denmark,26,4,6.500000
Egypt,19,3,6.333333
